<h1><center> Welcome to MotEx </center></h1>



# First import modules, set seed parameters and import functions

In [1]:
import numpy as np
import time
import matplotlib as mpl
import matplotlib.cm
from multiprocessing import Pool
from functools import partial

In [2]:
formfactor_dict = {
  "H": 1, "He": 2, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, "F": 9, "Ne": 10, "Na": 11, "Mg": 12,
  "Al": 13, "Si": 14, "P": 15, "S": 16, "Cl": 17, "Ar": 18, "K": 19, "Ca": 20, "Sc": 21, "Ti": 22, "V": 23, "Cr": 24,
  "Mn": 25, "Fe": 26, "Co": 27, "Ni": 28, "Cu": 29, "Zn": 30, "Ga": 31, "Ge": 32, "As": 33, "Se": 34, "Br": 35, "Kr": 36,
  "Rb": 37, "Sr": 38, "Y": 39, "Zr": 40, "Nb": 41, "Mo": 42, "Tc": 43, "Ru": 44, "Rh": 45, "Pd": 46, "Ag": 47, "Cd": 48,
  "In": 49, "Sn": 50, "Sb": 51, "Te": 52, "I": 53, "Xe": 54, "Cs": 55, "Ba": 56, "Hf": 72, "Ta": 73, "W": 74, "Re": 75,
  "Os": 76, "Ir": 77, "Pt": 78, "Au": 79, "Hg": 80, "Tl": 81, "Pb": 82, "Bi": 83, "Po": 84, "At": 85, "Rn": 86, "Fr": 87,
  "Ra": 88, "Rf": 104, "Db": 105, "Sg": 106, "Bh": 107, "Hs": 108, "Mt": 109, "Ds": 110, "Rg": 111, "Cn": 112, "Nh": 113, "Fl": 114,
  "Mc": 115, "Lv": 116, "Ts": 117, "Og": 118, "La": 57, "Ce": 58, "Pr": 59, "Nd": 60, "Pm": 61, "Sm": 62, "Eu": 63, "Gd": 64,
  "Tb": 65, "Dy": 66, "Ho": 67, "Er": 68, "Tm": 69, "Yb": 70, "Lu": 71, "Ac": 89, "Th": 90, "Pa": 91, "U": 92, "Np": 93,
  "Pu": 94, "Am": 95, "Cm": 96, "Bk": 97, "Cf": 98, "Es": 99, "Fm": 100, "Md": 101, "No": 102, "Lr": 103, "D": 1}

def Load(Experimental_Data, starting_model):
    """This function loads the data and structure"""
    
    # Get experimental data
    for skip_row in range(100):
        try:
            Exp_data = np.loadtxt(Experimental_Data, skiprows=skip_row)
        except ValueError:
            continue
    # Nyquist sampling the data by only using every 10nth datapoint
    if Exp_data[1,0] - Exp_data[0,0] < 0.1:
        Exp_r, Exp_Gr = Exp_data[::10,0], Exp_data[::10,1]
    else:
        Exp_r, Exp_Gr = Exp_data[:,0], Exp_data[:,1]
    # Normalise data
    Exp_Gr /= max(Exp_Gr)

    # Read structure and divide it into two lists: Atoms we want to iterate (W) and atoms we do not iterate (O)
    struct=[]
    with open(starting_model, 'r') as fi:
        for line in fi.readlines():
            sep_line=line.strip('{}\n\r ').split()
            if len(sep_line)==4: #  tillader andre informationer i xyz filen some ikke skal laeses
                struct.append(sep_line)
    elements=np.array(struct)[:,0]
    xyz=(np.array(struct)[:,1:].astype(float))
    
    return Exp_r, Exp_Gr, elements, xyz

def fitting(structure_catalogue, formfactor_dict, Exp_r, Exp_Gr, elements, xyz, index):    
    """This function takes in a 'starting_model', and an 'index' from the 'structure_catalogue'. It generates the 
    corresponding structure and calculate the Rwp value to the 'Experimental_Data without fitting"""
    
    # Cycle through W atoms and delete W according to index 0's from permutation
    delete_M = np.where(np.array(structure_catalogue)[index,:] == 0)[0]
 
    # Delete atoms from starting model 
    elements = np.delete(elements, delete_M, 0)
    xyz = np.delete(xyz, delete_M, 0)
    elements = [int(formfactor_dict[element]) for element in elements]

    # Calculate distances and formfactor to simulate PDF
    i, j = np.triu_indices(len(xyz), k=1)
    dists = np.sqrt((xyz[i,0]-xyz[j,0])**2+(xyz[i,1]-xyz[j,1])**2+(xyz[i,2]-xyz[j,2])**2)
    formfactor = np.array(elements)[i] * np.array(elements)[j]

    # Simulate PDF
    #Sim_Gr, Sim_r = torch.histogram(torch.tensor(dists).float(), bins=301, range=[-0.05,30.05], weight=torch.tensor(formfactor).float())
    Sim_Gr, Sim_r = np.histogram(np.array(dists, dtype=float), bins=len(Exp_r), range=[Exp_r[0]-0.05,Exp_r[-1]+0.05], weights=np.array(formfactor, dtype=float))
    Sim_r = (Sim_r[1:] + Sim_r[:-1]) / 2
    
    # Normalise PDF
    #Sim_Gr /= Exp_r
    Sim_Gr = np.nan_to_num(Sim_Gr, 0)
    Sim_Gr /= np.max(Sim_Gr)

    # Calculate Rwp value
    Rwp = np.sqrt(sum((Exp_Gr - Sim_Gr)**2) / sum((Exp_Gr)**2))
    #PCC = np.corrcoef(Exp_Gr[10:], Sim_Gr[10:])[0,1]

    return 1-Rwp

def fitting_multiprocess(structure_catalogue, formfactor_dict, Exp_r, Exp_Gr, element, xyz, SaveName, cores=1):
    """This function runs the calculations of all the structures in the structure catalogue using multiprocessing"""

    values = []
    # Set up multiprocessing refinement
    fitindex = range(len(structure_catalogue))
    p = Pool(processes=cores)
    func = partial(fitting, structure_catalogue, formfactor_dict, Exp_r, Exp_Gr, element, xyz)
    results = p.map(func, fitindex)
    p.close()
    p.join()
    
    # Start calculating residuals and append results to lists
    for i in fitindex:
        rw = results[i]
        values.append(rw)
    values = np.reshape(values,(int(len(values)/1) , 1))
    
    # Save results 
    Result = np.column_stack([values, np.asarray(structure_catalogue)])
    np.savetxt(SaveName, Result)
    return Result

def structure_catalogue_maker(Number_of_atoms):
    """Makes a catalogue of structures"""
    
    structure_catalogue = np.ones((Number_of_atoms,Number_of_atoms))
    structure_catalogue[np.array([range(Number_of_atoms)]),np.array([range(Number_of_atoms)])] = 0
    return structure_catalogue

def calculate_atomContributionValue(Result, saveResults):
    """Calculate atom contribution value list from the result array"""
    
    # Define AtomContributionValues vector
    AtomContributionValues = Result[:,0]
    
    # Normalise the AtomContributionValues
    amin, amax = min(AtomContributionValues), max(AtomContributionValues)
    AtomContributionValues = (AtomContributionValues - amin) / (amax - amin)
    
    # Define colormap of viridis.reverse
    norm = mpl.colors.Normalize(vmin=min(AtomContributionValues), vmax=max(AtomContributionValues))
    cmap = matplotlib.cm.cividis_r
    m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    
    # Save results to file
    f = open(saveResults+"AtomContributionValues.txt", "w")
    f.write("\nAtom contribution are calculated to: \n")
    for i in range(len(AtomContributionValues)):
        f.write("Atom # "+ str(i+1) + ":  "+ str(AtomContributionValues[i]) + "  Colorcode:  "+ mpl.colors.rgb2hex(m.to_rgba(AtomContributionValues[i]))+"\n")
    
    return m, AtomContributionValues

def Make_CrystalMakerFile(elements, xyz, AtomContributionValues, m, saveResults):
    # Output a crystalmaker file to visualize the results
    CrystalMaker = open(saveResults+'_CrystalMaker.cmtx', 'w')

    CrystalMaker.write("MOLE  CrystalMaker molecule format\n")
    CrystalMaker.write("TITL  Molecule\n\n")
    CrystalMaker.write("! Model type\n")
    CrystalMaker.write("MODL  1\n\n")

    CrystalMaker.write("! Depth fading settings\n")
    CrystalMaker.write("DCUE  1.000000 0.212899 0.704686\n\n")

    CrystalMaker.write("! Colour definitions:\n")
    CrystalMaker.write("TYPE\n")

    # Assign colors to all the atoms
    for iter, element in enumerate(elements):
        if iter < NumW:
            CrystalMaker.write(element + str(iter+1) + " 1.32 ")
            rgb1 = m.to_rgba(AtomContributionValues[iter])[:-1][0]
            rgb2 = m.to_rgba(AtomContributionValues[iter])[:-1][1]
            rgb3 = m.to_rgba(AtomContributionValues[iter])[:-1][2]
            CrystalMaker.write(str(rgb1) + " " + str(rgb2) + " " + str(rgb3))
            CrystalMaker.write("\n")
        else:
            CrystalMaker.write(element + str(iter+1) + " 0.66 ")
            rgb1 = mpl.colors.to_rgb("#FF0000")[0]
            rgb2 = mpl.colors.to_rgb("#FF0000")[1]
            rgb3 = mpl.colors.to_rgb("#FF0000")[2]
            CrystalMaker.write(str(rgb1) + " " + str(rgb2) + " " + str(rgb3))
            CrystalMaker.write("\n")
    
    CrystalMaker.write("\n")
    CrystalMaker.write("! Atoms list\n")
    CrystalMaker.write("! Bond Specifications\n")
    
    # Assign bonds between the atoms
    for iter, element in enumerate(elements):
        if iter < NumW:
            NI_elements = np.delete(np.unique(elements), np.where(np.unique(elements) == element)[0])
            for NI_element in NI_elements:
                CrystalMaker.write("BMAX " + element + str(iter+1) + " " + str(NI_element) + "  2.6")#    " O  2.6")
                CrystalMaker.write("\n")
    
    CrystalMaker.write("\n")
    CrystalMaker.write("! Atoms list\n")
    CrystalMaker.write("ATOM\n")
    
    # Assign coordinates to the atoms
    for iter, element in enumerate(elements):
        if iter < NumW:
            CrystalMaker.write(element + str(iter+1) + " " + element + str(iter+1) + " " + str(xyz[iter][0]) + " " + str(xyz[iter][1]) + " " + str(xyz[iter][2]) + "\n")
        else:
            CrystalMaker.write(element + " " + element + str(iter+1) + " " + str(xyz[iter][0]) + " " + str(xyz[iter][1]) + " " + str(xyz[iter][2]) + "\n")

    CrystalMaker.close()
    
    return None


# Define variables

In [3]:
starting_model = "Structure_Models/35643-ICSD_222unitcell.xyz" # Name of the starting model file
StemName = "Fe2O3_3204M_0"
NumW = 200 # Number of atoms that should be permuted in the starting model
saveResults = "Results/"

#starting_model = "Structure_Models/OLD_Paper/C60_111_supercell.xyz" # Name of the starting model file
#StemName = "C60_simulated"
#NumW = 132 # Number of atoms that should be permuted in the starting model

#starting_model = "Structure_Models/OLD_Paper/Mo36_Krebs_newest.xyz" # Name of the starting model file
#StemName = "Al-Mo15-rsub_dioptas"
#NumW = 36 # Number of atoms that should be permuted in the starting model

#starting_model = "Structure_Models/OLD_Paper/beta_Bi2O3_223_96atoms.xyz" # Name of the starting model file
#StemName = "Ligand_1-095summed"
#NumW = 96 # Number of atoms that should be permuted in the starting model

#starting_model = "Structure_Models/OLD_Paper/ABOCIE_real_111sc.xyz" # Name of the starting model file
#StemName = "DanMAX_AlphaKeggin"
#NumW = 48 # Number of atoms that should be permuted in the starting model

#starting_model = "Structure_Models/OLD_Paper/166877.xyz" # Name of the starting model file
#StemName = "DanMAX_AlphaKeggin"
#NumW = 72 # Number of atoms that should be permuted in the starting model

#starting_model = "Structure_Models/OLD_Paper/110457.xyz" # Name of the starting model file
#StemName = "DanMAX_AlphaKeggin"
#NumW = 48 # Number of atoms that should be permuted in the starting model

#starting_model = "Structure_Models/OLD_Paper/109725.xyz" # Name of the starting model file
#StemName = "DanMAX_AlphaKeggin"
#NumW = 24 # Number of atoms that should be permuted in the starting model

#starting_model = "Structure_Models/OLD_Paper/ABOCIE_real_111sc.xyz" # Name of the starting model file
#StemName = "AMWH2O"
#NumW = 48 # Number of atoms that should be permuted in the starting model

#starting_model = "Structure_Models/OLD_Paper/166877.xyz" # Name of the starting model file
#StemName = "AMWH2O"
#NumW = 72 # Number of atoms that should be permuted in the starting model

#starting_model = "Structure_Models/OLD_Paper/110457.xyz" # Name of the starting model file
#StemName = "AMWH2O"
#NumW = 48 # Number of atoms that should be permuted in the starting model

starting_model = "Structure_Models/OLD_Paper/109725.xyz" # Name of the starting model file
StemName = "AMWH2O"
NumW = 24 # Number of atoms that should be permuted in the starting model


# Run

In [4]:
start_time = time.time()
# Step 1: Make the structure catalogue
structure_catalogue = structure_catalogue_maker(Number_of_atoms=NumW)
print ("Catalogue maker: ", time.time() - start_time)

### First define the experimental data path and the path you want the structure catalogue with fits to be saved
Experimental_Data = "Experimental_Data/OLD_Paper/"+StemName+".gr" # Name of the experimental file
saveFits = "Training_Data/"+StemName+".txt" # Name of the saved fits file
# Load data and start model
Exp_r, Exp_Gr, elements, xyz = Load(Experimental_Data, starting_model)
print ("Data loader: ", time.time() - start_time)

### Step 2: Produce organized structure catalogue with Rwp values
Result = fitting_multiprocess(structure_catalogue, formfactor_dict, Exp_r, Exp_Gr, elements, xyz, SaveName=saveFits, cores=None)

print ("Getting results: ", time.time() - start_time)

# Step 3: Calculate Atom Contribution values
m, AtomContributionValues = calculate_atomContributionValue(Result, saveResults)
print ("Calculating atom contribution values: ", time.time() - start_time)

# Step 4: Output a CrystalMaker file
Make_CrystalMakerFile(elements, xyz, AtomContributionValues, m, saveResults+StemName)
print ("Output a CrystalMaker file: ", time.time() - start_time)

    

Catalogue maker:  0.00017213821411132812
Data loader:  0.023865938186645508
Getting results:  0.16741299629211426
Calculating atom contribution values:  0.17948007583618164
Output a CrystalMaker file:  0.19597101211547852
